In [ ]:
'''
Data Augmentation
1)Rotation
2)Noise
3)Horizontal Flip
'''

import random
from scipy import ndarray
import skimage as sk
from skimage import transform
from skimage import util

def random_rotation(image_array: ndarray):
    # pick a random degree of rotation between 25% on the left and 25% on the right
    random_degree = random.uniform(-25, 25)
    return sk.transform.rotate(image_array, random_degree)

def random_noise(image_array: ndarray):
    # add random noise to the image
    return sk.util.random_noise(image_array)

def horizontal_flip(image_array: ndarray):
    # horizontal flip doesn't need skimage, it's easy as flipping the image array of pixels !
    return image_array[:, ::-1]

In [ ]:
from __future__ import print_function
import numpy as np
np.random.seed(42)

class Layer:
    def __init__(self):
        self.weights = np.zeros(shape=(input.shape[1], 10))
        bias = np.zeros(shape=(10,))
        pass
    
    def forward(self, input):
        output = np.matmul(input, self.weights) + bias
        return output

class Dense(Layer):
    def __init__(self, input_units, output_units, learning_rate=0.1):
        self.learning_rate = learning_rate
        
        self.weights = np.random.randn(input_units, output_units)*0.01
        self.biases = np.zeros(output_units)
        
    def forward(self,input):
        return np.matmul(input, self.weights) + self.biases
      
    def backward(self,input,grad_output):
        grad_input = np.dot(grad_output,np.transpose(self.weights))#Gradient wrt to input
        grad_weights = np.transpose(np.dot(np.transpose(grad_output),input))#Gradient w.r.t. weights 
        grad_biases = np.sum(grad_output, axis = 0)#Gradient w.r.t. biases
        #Weight updation
        self.weights = self.weights - self.learning_rate * grad_weights
        self.biases = self.biases - self.learning_rate * grad_biases
        return grad_input

'''
Activation Functions

1)Sigmoid
2)Relu
3)LRelu
4)Tanhh
5)Linear

'''   
    
class Sigmoid(Layer):
    def __init__(self):
        '''ReLU layer simply applies elementwise rectified linear unit to all inputs'''
        self.input = None
        self.output = None
        pass
    
    def forward(self, input):
        '''Apply elementwise ReLU to [batch, input_units] matrix'''
        self.input = input
        self.output = (1 / (1 + np.exp(-input)))
        return self.output

    def backward(self, input, grad_output):
        '''Compute gradient of loss w.r.t. ReLU input'''
        sigmoid_grad = (1 - self.output) * self.output
        return (grad_output * sigmoid_grad)
    
    
class LReLU(Layer):
    def __init__(self):
        '''ReLU layer simply applies elementwise rectified linear unit to all inputs'''
        pass
    
    def forward(self, input):
        '''Apply elementwise ReLU to [batch, input_units] matrix'''
        return np.maximum(-input,input)

    def backward(self, input, grad_output):
        '''Compute gradient of loss w.r.t. ReLU input'''
        lrelu_grad = []
#         print(input.shape)
        for i in input:
            lrelu_grad1=[]
            for j in i:
                lrelu_grad1.append(1 if j > 0 else -1)
            lrelu_grad.append(lrelu_grad1)
        lrelu_grad = np.array(lrelu_grad)
#         print(grad_output.shape,lrelu_grad.shape)
        return grad_output*lrelu_grad 

class ReLU(Layer):
    def __init__(self):
        pass
    
    def forward(self, input):
        return np.maximum(0,input)

    def backward(self, input, grad_output):
        relu_grad = input > 0
        return grad_output*relu_grad 

    
class Tanh(Layer):
    def __init__(self):
        self.output = None
        pass
    def forward(self,input):
        self.input = input
        self.output = (np.exp(input) - np.exp(-input))/(np.exp(input) + np.exp(-input))
        return self.output
    def backward(self,input,grad_output):
        return (1 - np.square(self.output))*grad_output
    
class Linear(Layer):
    def __init__(self):
        self.output = None
        pass
    def forward(self,input):
        self.input = input
        self.output = input
        return self.output
    def backward(self,input,grad_output):
        return 1 * grad_output
        
class softmax(Layer):
    def __init__(self):
        self.input = None
        self.output = None
        pass
    def forward(self,input):
        self.input = input
        self.output = np.exp(predicted) / np.exp(predicted).sum(axis=-1,keepdims=True)
        return self.output
    def backward(self,input,grad_output):
        return self.output*(1-self.output)*grad_output
        
''' Loss Functions and their gradients 

1) Logloss
2) CrossEntropy

'''

from keras.utils import to_categorical

'''
Here if we use argmax in the predict function we cant use logloss
So we have to implement another kind of soft max instaed of argmax
'''
def logloss_with_predicted(predicted,reference_answers):
    reference_answers = to_categorical(list(reference_answers),num_classes=10)
    reference_answers = np.array(reference_answers)
#     print(predicted.shape,reference_answers.shape)
#     print(predicted[0],reference_answers[0])
    loss = reference_answers * np.log(predicted) + (1-reference_answers) * np.log(1-predicted)
    return loss    

def grad_logloss_with_predicted(predicted,reference_answers):
    reference_answers = to_categorical(list(reference_answers),num_classes=10)
    reference_answers = np.array(reference_answers)
#     print(predicted.shape,reference_answers.shape)
#     print(predicted[0],reference_answers[0])
    return (predicted - reference_answers)
    
def softmax_crossentropy_with_predicted(predicted,reference_answers):
    predicted_for_answers = predicted[np.arange(len(predicted)),reference_answers]
#     print(reference_answers[0])
    xentropy = - predicted_for_answers + np.log(np.sum(np.exp(predicted),axis=-1))
#     print(xentropy)
    return xentropy

def grad_softmax_crossentropy_with_predicted(predicted,reference_answers):
    ones_for_answers = np.zeros_like(predicted)
    ones_for_answers[np.arange(len(predicted)),reference_answers] = 1
    
    softmax = np.exp(predicted) / np.exp(predicted).sum(axis=-1,keepdims=True)
    
    return (- ones_for_answers + softmax) / predicted.shape[0]
  


''' Load the mnist data set '''

import keras
def load_dataset(flatten=True,augment = True):
    if augment==False:
        (X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

        # normalize x
        X_train = X_train.astype(float) / 255.
        X_test = X_test.astype(float) / 255.

        # we reserve the last 10000 training examples for validation
        X_train, X_val = X_train[:-10000], X_train[-10000:]
        y_train, y_val = y_train[:-10000], y_train[-10000:]

        if flatten:
            X_train = X_train.reshape([X_train.shape[0], -1])
            X_val = X_val.reshape([X_val.shape[0], -1])
            X_test = X_test.reshape([X_test.shape[0], -1])

        return X_train, y_train, X_val, y_val, X_test, y_test
    else:
        (X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
        X_train = X_train.astype(float) / 255.
        X_test = X_test.astype(float) / 255.
        
        idx = random.sample(range(0,len(X_train)),60)
#         print(random_rotation(np.array(X_train[idx[:20],:])).shape)
        X_train = np.append(X_train,random_rotation(np.array(X_train[idx[:20],:])),axis=0)
        y_train = np.append(y_train,y_train[idx[:20]],axis=0)
        X_train = np.append(X_train,random_noise(np.array(X_train[idx[20:40],:])),axis=0)
        y_train = np.append(y_train,y_train[idx[20:40]],axis=0)
        X_train = np.append(X_train,random_rotation(np.array(X_train[idx[40:60],:])),axis=0)
        y_train = np.append(y_train,y_train[idx[40:60]],axis=0)
        X_train, X_val = X_train[:-10000], X_train[-10000:]
        y_train, y_val = y_train[:-10000], y_train[-10000:]

        if flatten:
            X_train = X_train.reshape([X_train.shape[0], -1])
            X_val = X_val.reshape([X_val.shape[0], -1])
            X_test = X_test.reshape([X_test.shape[0], -1])

        return X_train, y_train, X_val, y_val, X_test, y_test
        
        
        

import matplotlib.pyplot as plt
X_train, y_train, X_val, y_val, X_test, y_test = load_dataset(flatten=True)

plt.figure(figsize=[6,6])
for i in range(4):
    plt.subplot(2,2,i+1)
    plt.title("Label: %i"%y_train[i])
    plt.imshow(X_train[i].reshape([28,28]),cmap='gray');
    
network = []
network.append(Dense(X_train.shape[1],100))
network.append(ReLU())
network.append(Dense(100,200))
network.append(ReLU())
network.append(Dense(200,50))
network.append(ReLU())
network.append(Dense(50,10))

def forward(network, X):
    
    activations = []
    input = X
    for i in range(len(network)):
        activations.append(network[i].forward(X))
        X = network[i].forward(X)
        
    assert len(activations) == len(network)
    return activations

def predict(network,X,conf=False):
    
    predicted = forward(network,X)[-1]
    '''
    WITHOUT CONFIDENCE SCORES
    '''
    if conf == False:
        return predicted.argmax(axis=-1)
    else:
        pass

def train(network,X,y,regularize='L1Norm',alp=0.01):
    
    # Get the layer activations
    layer_activations = forward(network,X)
    predicted = layer_activations[-1]
#     alp = int(input('Enter alpha for regularizer'))
    # Compute the loss and the initial gradient
    loss = softmax_crossentropy_with_predicted(predicted,y)
    
#     loss = logloss_with_predicted(predicted,y)
#     print(loss.shape)
    '''
    Regularizer
    1)L1Norm
    2)L2Norm
    '''
    regularizer = 0
    if regularize == 'L1Norm':
        for i in range(0,len(network),2):
            regularizer += abs(np.mean(network[i].weights))
    elif regularize=='L2Norm':
        for i in range(0,len(network),2):
            regularizer += (np.mean(network[i].weights) ** 2)
    loss += alp * regularizer
    loss_grad = grad_softmax_crossentropy_with_predicted(predicted,y)
#     loss_grad = grad_logloss_with_predicted(predicted,y)
#     print('1',loss_grad.shape)
    for i in range(1, len(network)):
        loss_grad = network[len(network) - i].backward(layer_activations[len(network) - i - 1], loss_grad)
    
    return np.mean(loss)
  
from tqdm import trange
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.random.permutation(len(inputs))
    for start_idx in trange(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]
        
train_log = []
val_log = []
for epoch in range(10):

    for x_batch,y_batch in iterate_minibatches(X_train,y_train,batchsize=32,shuffle=True):
        train(network,x_batch,y_batch)
    
    train_log.append(np.mean(predict(network,X_train)==y_train))
    val_log.append(np.mean(predict(network,X_val)==y_val))
    
#     clear_output()
    print("Epoch",epoch)
    print("Train accuracy:",train_log[-1])
    print("Val accuracy:",val_log[-1])
    plt.plot(train_log,label='train accuracy')
    plt.plot(val_log,label='val accuracy')
    plt.legend(loc='best')
    plt.grid()
    plt.show()